In [36]:
import os
from Code.Utils import utils
from Code.Charts import charts
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import TargetEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (c:\Users\wren1\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [37]:
! conda install -c glemaitre imbalanced-learn

# Feature Engineering

Steps:

1. Read in data
2. Clean data
3. Encode features
4. Feature selection

In [22]:
## Read in data 
print(os.getcwd())
full_df = pd.read_csv('data/sbdb_query_results.csv')
full_df.info(verbose=True, show_counts=True)

c:\Users\wren1\Documents\theory of machine learning\EN625.742_group_project\src
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35188 entries, 0 to 35187
Data columns (total 73 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   full_name       35188 non-null  object 
 1   a               35188 non-null  float64
 2   e               35188 non-null  float64
 3   i               35188 non-null  float64
 4   om              35188 non-null  float64
 5   w               35188 non-null  float64
 6   q               35188 non-null  float64
 7   ad              35188 non-null  float64
 8   per_y           35188 non-null  float64
 9   data_arc        34876 non-null  float64
 10  condition_code  35187 non-null  float64
 11  n_obs_used      35188 non-null  int64  
 12  n_del_obs_used  1030 non-null   float64
 13  n_dop_obs_used  1030 non-null   float64
 14  H               35183 non-null  float64
 15  epoch_mjd       35188 non-null  int64  
 

C:\Users\wren1\AppData\Local\Temp\ipykernel_7388\3398585979.py:3: DtypeWarning: Columns (18,25,26,29,61) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('data/sbdb_query_results.csv')


In [30]:
# Clean data

# drop any colums with more than 20% missing values
df = utils.filterLowDataColumns(full_df, 0.2)

# drop any rows with missing values
df = df.dropna()

# df.info(verbose=True, show_counts=True)

# # drop any columns that are not useful for the model, like the producer of the record,
# the time of observation, and the standard error of existing columns
cols_to_drop = ['epoch_mjd', 
                'epoch', 
                'epoch_cal', 
                'first_obs', 
                'last_obs', 
                'producer',
                'sigma_e',
                'sigma_a',
                'sigma_q',
                'sigma_i',
                'sigma_om',
                'sigma_w',
                'sigma_ma',
                'sigma_ad',
                'sigma_n',
                'sigma_tp',
                'sigma_per',
                'rms',
                'full_name',
                'pdes',
                'condition_code',
                'tp_cal',
                'neo',
                'equinox',
                'sats']


df = df.drop(columns=cols_to_drop)

# drop spkid column - unique identifier
df = df.drop(columns=['spkid'])

# reset index to match the number of rows
df = df.reset_index(drop=True)

# separate target variable from features
target = df['pha']
features = df.drop(columns=['pha'])


# Scale numerical columns with RobustScaler (since many columns are not normally distributed)
num_cols = features.select_dtypes(include=['float64','int64']).columns

# Create a pipeline to scale numerical columns
num_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

# Fit and transform numerical columns
features[num_cols] = num_pipeline.fit_transform(features[num_cols])

# Encode categorical variables
categorical_cols = features.select_dtypes(include=['object']).columns

# Create a pipeline to encode categorical columns
cat_pipeline = Pipeline([
    ('encoder', TargetEncoder())
])

# Fit and transform categorical columns
features[categorical_cols] = cat_pipeline.fit_transform(features[categorical_cols], target)

# check if transformations and encoding worked
# print(features.info())
print(features.info())


# check for class imbalance
print(target.value_counts())

# fix class imbalance by oversampling the minority class

ros = RandomOverSampler(random_state=0)
features_resampled, target_resampled = ros.fit_resample(features, target)

# check if oversampling worked
print(target_resampled.value_counts())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34847 entries, 0 to 34846
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   a           34847 non-null  float64
 1   e           34847 non-null  float64
 2   i           34847 non-null  float64
 3   om          34847 non-null  float64
 4   w           34847 non-null  float64
 5   q           34847 non-null  float64
 6   ad          34847 non-null  float64
 7   per_y       34847 non-null  float64
 8   data_arc    34847 non-null  float64
 9   n_obs_used  34847 non-null  float64
 10  H           34847 non-null  float64
 11  ma          34847 non-null  float64
 12  n           34847 non-null  float64
 13  tp          34847 non-null  float64
 14  per         34847 non-null  float64
 15  moid        34847 non-null  float64
 16  moid_ld     34847 non-null  float64
 17  moid_jup    34847 non-null  float64
 18  t_jup       34847 non-null  float64
 19  class       34847 non-nul